In [ ]:
import os
import numpy as np
import pandas as pd
import scipy as scipy
from tqdm import tqdm
import matplotlib.pyplot as plt

import mar
import preprocessing

In [2]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), "data")
pat_num = 4

In [3]:
mar4 = preprocessing.load_mar_data(data_dir, pat_num)
sickbay4 = preprocessing.load_sickbay_data(data_dir, pat_num)

In [ ]:
print(sickbay4.head())

In [4]:
sbs4 = preprocessing.load_retro_data(data_dir, pat_num, ignore_default=True)
sbs4 = sbs4[['time', 'MD_SBS', 'TM_SBS', 'SBS']]

In [ ]:
print(mar4)
print(sickbay4)
print(sbs4)

In [ ]:
raw_data = preprocessing.load_mat_file(os.path.join(data_dir, "Patient4", "Patient4_SICKBAY_15MIN_1MIN_Retro.mat"))

for key in raw_data.keys():
    if raw_data[key].ndim == 2:
        new = np.empty(raw_data[key].shape[0], dtype=object)

        for j in range(len(new)):
            new[j] = raw_data[key]
        
        raw_data[key] = new

sickbay_df = pd.DataFrame(raw_data)
display(sickbay_df.head())

In [ ]:
accel4 = preprocessing.load_accel_data(data_dir, pat_num)

In [ ]:
accel4.to_csv(os.path.join(data_dir, "Patient4", "accel_raw.csv"), index=False)

In [ ]:
accel4 = accel4.sort_values(by='time', ascending=True).reset_index(drop=True)

In [ ]:
print(accel4.head())

In [ ]:
accel4_downsampled = accel4.iloc[::200].reset_index(drop=True)
accel4_downsampled['time'] = accel4_downsampled['time'].dt.round('S')

In [ ]:
print(accel4_downsampled.shape)
print(accel4_downsampled.head())

In [ ]:
accel4_downsampled['a'] = np.sqrt(accel4_downsampled['X']**2 + accel4_downsampled['Y']**2 + accel4_downsampled['Z']**2)
accel4_downsampled.to_csv(os.path.join(data_dir, "Patient4", "accel_downsampled.csv"), index=False)

In [ ]:
mar4_narcotics, mar4_paralytics, mar4_alpha_agonists, mar4_ketamines, mar4_propofols, mar4_etomidates, mar4_benzodiazepines = mar.filter_mar(mar4)

print(mar4_narcotics.shape)
print(mar4_paralytics.shape)
print(mar4_alpha_agonists.shape)
print(mar4_ketamines.shape)
print(mar4_propofols.shape)
print(mar4_etomidates.shape)
print(mar4_benzodiazepines.shape)

In [ ]:
narcotics_half_lives = {'fentanyl': 4.5, 'hydromorphone': 2.5, 'methadone': 19.5}
narcotics_elimination_rates = {med: np.log(2) / (half_life * 60) for med, half_life in narcotics_half_lives.items()}

mar4_fentanyl = mar4_narcotics[mar4_narcotics['med_name'].str.contains('fentanyl', case=False)]

fentanyl_doses = mar.calculate_doses(mar4_fentanyl)

fentanyl_concetrations = mar.calculate_concentrations(fentanyl_doses, narcotics_elimination_rates['fentanyl'])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(fentanyl_concetrations['time'], fentanyl_concetrations['concentration'], label='Fentanyl Concentration')
plt.xlabel('Time')
plt.ylabel('Concentration (ug/kg)')
plt.title('Fentanyl Concentration Over Time')
plt.legend()
plt.grid()
plt.show()

In [ ]:
start = sbs4['time'].iloc[1]
end = sbs4['time'].iloc[-1]

plt.figure(figsize=(10, 6))
plt.plot(fentanyl_concetrations['time'], fentanyl_concetrations['concentration'], label='Fentanyl Concentration')
for i in range(1, len(sbs4)):
    if sbs4['TM_SBS'].iloc[i] == -2:
        plt.axvline(x=sbs4['time'].iloc[i], color='blue', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == -1:
        plt.axvline(x=sbs4['time'].iloc[i], color='green', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == 0:
        plt.axvline(x=sbs4['time'].iloc[i], color='yellow', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == 1:
        plt.axvline(x=sbs4['time'].iloc[i], color='orange', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == 2:
        plt.axvline(x=sbs4['time'].iloc[i], color='red', linestyle='--')
plt.xlim(start, end)
plt.title('Fentanyl Concentration and Dose vs Time with SBS')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
mar4_dexmedetomidine = mar4_alpha_agonists[mar4_alpha_agonists['med_name'].str.contains('dexmedetomidine', case=False)]
mar4_clonidine = mar4_alpha_agonists[mar4_alpha_agonists['med_name'].str.contains('clonidine', case=False)]

alpha_agonists_half_lives = {'dexmedetomidine': 2, 'clonidine': 1.25}
alpha_agonists_elimination_rates = {med: np.log(2) / (half_life * 60) for med, half_life in alpha_agonists_half_lives.items()}

dexmedetomidine_doses = mar.calculate_doses(mar4_dexmedetomidine)
clonidine_doses = mar.calculate_doses(mar4_clonidine)

dexmedetomidine_concentrations = mar.calculate_concentrations(dexmedetomidine_doses, alpha_agonists_elimination_rates['dexmedetomidine'])
clonidine_concentrations = mar.calculate_concentrations(clonidine_doses, alpha_agonists_elimination_rates['clonidine'])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(dexmedetomidine_concentrations['time'], dexmedetomidine_concentrations['concentration'], label='Dexmedetomidine Concentration')
plt.plot(clonidine_concentrations['time'], clonidine_concentrations['concentration'], label='Clonidine Concentration')
plt.xlabel('Time')
plt.ylabel('Concentration (ug/kg)')
plt.title('Alpha Agonists Concentration Over Time')
plt.legend()
plt.grid()
plt.show()

In [ ]:
start = sbs4['time'].iloc[1]
end = sbs4['time'].iloc[-1]

plt.figure(figsize=(10, 6))
plt.plot(dexmedetomidine_concentrations['time'], dexmedetomidine_concentrations['concentration'], label='Dexmedetomidine')
plt.plot(clonidine_concentrations['time'], clonidine_concentrations['concentration'], label='Clonidine')
for i in range(1, len(sbs4)):
    if sbs4['TM_SBS'].iloc[i] == -2:
        plt.axvline(x=sbs4['time'].iloc[i], color='blue', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == -1:
        plt.axvline(x=sbs4['time'].iloc[i], color='green', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == 0:
        plt.axvline(x=sbs4['time'].iloc[i], color='yellow', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == 1:
        plt.axvline(x=sbs4['time'].iloc[i], color='orange', linestyle='--')
    elif sbs4['TM_SBS'].iloc[i] == 2:
        plt.axvline(x=sbs4['time'].iloc[i], color='red', linestyle='--')
plt.xlim(start, end)
plt.title('Alpha Agonist Concentration and Dose vs Time with SBS')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
hr4 = sickbay4[['time', 'heart_rate']]
rr4 = sickbay4[['time', 'respiratory_rate']]

In [ ]:
start = hr4['time'].iloc[0]
end = hr4['time'].iloc[-1]

accel4_downsampled = accel4_downsampled[['time', 'a']]

accel4_downsampled = accel4_downsampled[accel4_downsampled['time'].between(start, end)]
accel4_standardized = accel4_downsampled.copy()
accel4_standardized['a'] = (accel4_downsampled['a'] - accel4_downsampled['a'].mean()) / accel4_downsampled['a'].std()

hr4 = hr4[hr4['time'].between(start, end)]
hr4_standardized = hr4.copy()
hr4_standardized['heart_rate'] = (hr4['heart_rate'] - hr4['heart_rate'].mean()) / hr4['heart_rate'].std()

rr4= rr4[rr4['time'].between(start, end)]
rr4_standardized = rr4.copy()
rr4_standardized['respiratory_rate'] = (rr4['respiratory_rate'] - rr4['respiratory_rate'].mean()) / rr4['respiratory_rate'].std()

chunk_size = 10000

for start_idx in range(0, len(hr4_standardized), chunk_size):
    end_idx = start_idx + chunk_size

    accel_chunk = accel4_standardized.iloc[start_idx:end_idx]
    hr_chunk    = hr4_standardized.iloc[start_idx:end_idx]
    rr_chunk    = rr4_standardized.iloc[start_idx:end_idx]

    plt.figure(figsize=(10, 6))
    plt.plot(accel_chunk['time'], accel_chunk['a'],              label='Acceleration (z)')
    plt.plot(hr_chunk['time'],    hr_chunk['heart_rate'],        label='Heart Rate (z)')
    plt.plot(rr_chunk['time'],    rr_chunk['respiratory_rate'],  label='Respiratory Rate (z)')
    
    window_start = accel_chunk['time'].iloc[0]
    window_end   = accel_chunk['time'].iloc[-1]
    events_in_window = sbs4[sbs4['time'].between(window_start, window_end)]
    color_map = {-2:'blue', -1:'green', 0:'yellow', 1:'orange', 2:'red'}
    for _, row in events_in_window.iterrows():
        plt.axvline(x=row['time'], color=color_map[row['TM_SBS']], linestyle='--')

    plt.xlim(window_start, window_end)
    plt.ylim(-15, 15)
    plt.xlabel('Time')
    plt.ylabel('Standardized z‑score')
    plt.title(f'Standardized Features: points {start_idx}–{min(end_idx,len(hr4_standardized))}')
    plt.legend()
    plt.grid()
    plt.show()


Hello World!

In [ ]:
def binary_search(times, target, start, end):
    if start > end:
        return -1

    mid = (start + end) // 2

    if times[mid] == target:
        return mid
    elif times[mid] < target:
        return binary_search(times, target, mid + 1, end)
    else:
        return binary_search(times, target, start, mid - 1)

In [ ]:
start = 0
end = len(accel4) - 1

indices = np.zeros(len(sbs4), dtype=int)
chunks = []

for i in range(len(sbs4)):
    index = binary_search(accel4['time'].values, sbs4['time'].values[i], start, end)
    indices[i] = index

    if index != -1:
        chunks.append(tuple([index - 100*60*15, index + 100*60*15]))
    else:
        chunks.append(tuple([0, 0]))

In [ ]:
accel4_chunks = pd.DataFrame(columns=['time', 'accel_x', 'accel_y', 'accel_z'])

for i in tqdm(range(len(chunks))):
    if chunks[i][0] < 0:
        continue

    if chunks[i][1] >= len(accel4):
        continue

    time = accel4['time'].values[chunks[i][0]:chunks[i][1]]
    accelx = accel4['X'].values[chunks[i][0]:chunks[i][1]]
    accely = accel4['Y'].values[chunks[i][0]:chunks[i][1]]
    accelz = accel4['Z'].values[chunks[i][0]:chunks[i][1]]

    accel4_chunks = pd.concat([accel4_chunks, pd.DataFrame({'time': time, 'accel_x': accelx, 'accel_y': accely, 'accel_z': accelz})], ignore_index=True)
    accel4_chunks = accel4_chunks.sort_values(by='time', ascending=True).reset_index(drop=True)